<a href="https://colab.research.google.com/github/chadloh/GoogleDecimeterChallenge/blob/main/SatellitePath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pymap3d pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import math
import numpy as np
import pandas as pd
import pymap3d as pm
import pymap3d.vincenty as pmv
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import glob as gl 
import pyproj as proj
import warnings
from tqdm.auto import tqdm
from pprint import pprint

import scipy.optimize
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.spatial import distance
from scipy import signal
from scipy.ndimage.interpolation import shift

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# Constants
CLIGHT = 299_792_458   # speed of light (m/s)
RE_WGS84 = 6_378_137   # earth semimajor axis (WGS84) (m)
OMGE = 7.2921151467E-5  # earth angular velocity (IS-GPS) (rad/s)

In [118]:
path = '/content/drive/MyDrive/Google_decimeter/smartphone-decimeter-2022/'

gnss_df_combined = pd.DataFrame()
# Loop for each trip
for i, dirname in enumerate(tqdm(sorted(gl.glob(f'{path}/train/*/*/')))):
  

  drive, phone = dirname.split('/')[-3:-1]
  tripID = f'{drive}/{phone}'
  print(tripID)

  if phone != 'GooglePixel4': continue

  # Read data
  gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv')  # GNSS data
  # gt_test = pd.read_csv(f'{dirname}/ground_truth.csv')  # ground truth
  gnss_df['SVID'] = list(map(lambda x,y: str(x)+'_'+str(y)+'_'+str(i), gnss_df['SignalType'], gnss_df['Svid']))

  gnss_df_combined = pd.concat([gnss_df_combined, gnss_df], axis = 0)

  # pd.DataFrame(x_wls).to_csv(f'{dirname}/x_wls_1.csv', index=False)

  0%|          | 0/170 [00:00<?, ?it/s]

2020-05-15-US-MTV-1/GooglePixel4XL
2020-05-21-US-MTV-1/GooglePixel4
2020-05-21-US-MTV-1/GooglePixel4XL
2020-05-21-US-MTV-2/GooglePixel4
2020-05-21-US-MTV-2/GooglePixel4XL
2020-05-28-US-MTV-2/GooglePixel4
2020-05-28-US-MTV-2/GooglePixel4XL
2020-05-29-US-MTV-1/GooglePixel4
2020-05-29-US-MTV-1/GooglePixel4XL
2020-05-29-US-MTV-2/GooglePixel4
2020-05-29-US-MTV-2/GooglePixel4XL
2020-06-04-US-MTV-1/GooglePixel4
2020-06-04-US-MTV-1/GooglePixel4XL
2020-06-04-US-MTV-2/GooglePixel4
2020-06-04-US-MTV-2/GooglePixel4XL
2020-06-05-US-MTV-1/GooglePixel4
2020-06-05-US-MTV-1/GooglePixel4XL
2020-06-05-US-MTV-2/GooglePixel4
2020-06-05-US-MTV-2/GooglePixel4XL
2020-06-10-US-MTV-1/GooglePixel4
2020-06-10-US-MTV-1/GooglePixel4XL
2020-06-10-US-MTV-2/GooglePixel4
2020-06-10-US-MTV-2/GooglePixel4XL
2020-06-11-US-MTV-1/GooglePixel4
2020-06-11-US-MTV-1/GooglePixel4XL
2020-06-18-US-MTV-1/GooglePixel4
2020-06-18-US-MTV-1/GooglePixel4XL
2020-06-24-US-MTV-1/GooglePixel4
2020-06-24-US-MTV-1/GooglePixel4XL
2020-06-24-US

In [115]:
gnss_df_combined[gnss_df_combined.SVID.str.contains("L1_5")][::20]

,MessageType,utcTimeMillis,TimeNanos,LeapSecond,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond,IsrbMeters,IonosphericDelayMeters,TroposphericDelayMeters,WlsPositionXEcefMeters,WlsPositionYEcefMeters,WlsPositionZEcefMeters,SVID
72683,Raw,1589576386445,12532908000000,NaN,-1273599071537528615,0.151148,18.795930,-11.217746,9.496156,7,...,2327.595257,-3810.088991,-0.000226,0.0,13.334761,44.357782,-2.710762e+06,-4.277021e+06,3.865309e+06,GPS_L1_5_0
77051,Raw,1589576547445,12693908000000,NaN,-1273599071537525786,0.673042,21.251091,55.680012,9.710142,7,...,2377.375840,-3810.126508,-0.000240,0.0,13.091757,37.046025,-2.712086e+06,-4.273516e+06,3.868230e+06,GPS_L1_5_0
82490,Raw,1589576753445,12899908000000,NaN,-1273599071537519780,0.377445,24.093366,29.427179,11.722095,7,...,2439.181902,-3810.237484,-0.000257,0.0,12.777647,30.233212,-2.712280e+06,-4.269506e+06,3.872304e+06,GPS_L1_5_0
87524,Raw,1589576945445,13091908000000,NaN,-1273599071537518741,0.137392,18.180128,-6.952702,10.734965,7,...,2494.831378,-3810.288454,-0.000274,0.0,12.473098,25.445086,-2.712284e+06,-4.268682e+06,3.873190e+06,GPS_L1_5_0
88255,Raw,1589576969445,13115908000000,NaN,-1273599071537518734,0.468763,21.225856,10.335055,11.933077,7,...,2501.652920,-3810.295046,-0.000276,0.0,12.434615,24.933204,-2.712033e+06,-4.268760e+06,3.873251e+06,GPS_L1_5_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92090,Raw,1615923228000,6647570000000,NaN,-1299951798430257293,0.000000,7.373674,NaN,NaN,0,...,-3117.873424,-10912.924758,0.000002,0.0,8.328166,14.986398,-2.704345e+06,-4.288722e+06,3.856663e+06,GPS_L1_5_95
93012,Raw,1615923248000,6667570000000,NaN,-1299951798430257616,0.000000,8.392608,NaN,NaN,0,...,-3119.181536,-10912.924712,0.000003,0.0,8.352066,15.103646,-2.704220e+06,-4.288797e+06,3.856679e+06,GPS_L1_5_95
94162,Raw,1615923273000,6692570000000,NaN,-1299951798430258007,0.000000,8.255799,NaN,NaN,0,...,-3120.779622,-10912.924613,0.000005,0.0,8.381735,15.250758,-2.704187e+06,-4.288804e+06,3.856683e+06,GPS_L1_5_95
95071,Raw,1615923293000,6712570000000,NaN,-1299951798430258335,0.000000,7.016226,NaN,NaN,0,...,-3122.028447,-10912.924500,0.000006,0.0,8.405513,15.370305,-2.704195e+06,-4.288812e+06,3.856683e+06,GPS_L1_5_95


In [120]:
import plotly.express as px

fig = px.line_3d(gnss_df_combined[gnss_df_combined.SVID.str.contains("G1_24")][::20], x="SvPositionXEcefMeters", y="SvPositionYEcefMeters", z="SvPositionZEcefMeters", color='SVID')
fig.update_layout(
    autosize=False,
    width=1600,
    height=1600,)
fig.show()


In [ ]:
LLA = pd.DataFrame(
    np.array(pm.ecef2geodetic(gnss_df['WlsPositionXEcefMeters'], gnss_df['WlsPositionYEcefMeters'], gnss_df['WlsPositionZEcefMeters'])).T,
    columns = ['lat', 'lon', 'alt'])


pd.concat([gnss_df, LLA], axis = 1)


,MessageType,utcTimeMillis,TimeNanos,LeapSecond,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,SvClockDriftMetersPerSecond,IsrbMeters,IonosphericDelayMeters,TroposphericDelayMeters,WlsPositionXEcefMeters,WlsPositionYEcefMeters,WlsPositionZEcefMeters,lat,lon,alt
0,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.001592,0.000000,4.733301,3.394811,-2.693907e+06,-4.297452e+06,3.854203e+06,37.416664,-122.082013,-31.511228
1,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.002044,0.000000,3.933861,2.831474,-2.693907e+06,-4.297452e+06,3.854203e+06,37.416664,-122.082013,-31.511228
2,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.002146,0.000000,5.685666,4.716946,-2.693907e+06,-4.297452e+06,3.854203e+06,37.416664,-122.082013,-31.511228
3,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,0.003038,0.000000,5.565332,4.452569,-2.693907e+06,-4.297452e+06,3.854203e+06,37.416664,-122.082013,-31.511228
4,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,0.000590,0.000000,4.065440,3.035621,-2.693907e+06,-4.297452e+06,3.854203e+06,37.416664,-122.082013,-31.511228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90148,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,0.000022,-2351.409293,12.279992,5.939868,-2.711797e+06,-4.268995e+06,3.873143e+06,37.631686,-122.424968,-10.568773
90149,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,0.000788,-2348.542977,6.990076,2.671303,-2.711797e+06,-4.268995e+06,3.873143e+06,37.631686,-122.424968,-10.568773
90150,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,-0.000428,-2348.542977,16.376517,8.686505,-2.711797e+06,-4.268995e+06,3.873143e+06,37.631686,-122.424968,-10.568773
90151,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,-0.009129,-2348.542977,7.696385,3.070332,-2.711797e+06,-4.268995e+06,3.873143e+06,37.631686,-122.424968,-10.568773


In [ ]:
pd.merge(gnss_df, 
         gt_test[['UnixTimeMillis', 'LatitudeDegrees', 'LongitudeDegrees']], 
         how = 'left',
         left_on = 'utcTimeMillis',
         right_on = 'UnixTimeMillis')


,MessageType,utcTimeMillis,TimeNanos,LeapSecond,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,...,SvClockDriftMetersPerSecond,IsrbMeters,IonosphericDelayMeters,TroposphericDelayMeters,WlsPositionXEcefMeters,WlsPositionYEcefMeters,WlsPositionZEcefMeters,UnixTimeMillis,LatitudeDegrees,LongitudeDegrees
0,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.001592,0.000000,4.733301,3.394811,-2.693907e+06,-4.297452e+06,3.854203e+06,1589573679445,37.416619,-122.082065
1,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.002044,0.000000,3.933861,2.831474,-2.693907e+06,-4.297452e+06,3.854203e+06,1589573679445,37.416619,-122.082065
2,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,-0.002146,0.000000,5.685666,4.716946,-2.693907e+06,-4.297452e+06,3.854203e+06,1589573679445,37.416619,-122.082065
3,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,0.003038,0.000000,5.565332,4.452569,-2.693907e+06,-4.297452e+06,3.854203e+06,1589573679445,37.416619,-122.082065
4,Raw,1589573679445,9825908000000,NaN,-1273599071537525599,0.414394,29.760322,0.058358,10.712862,6,...,0.000590,0.000000,4.065440,3.035621,-2.693907e+06,-4.297452e+06,3.854203e+06,1589573679445,37.416619,-122.082065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90148,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,0.000022,-2351.409293,12.279992,5.939868,-2.711797e+06,-4.268995e+06,3.873143e+06,1589577040445,37.631667,-122.424972
90149,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,0.000788,-2348.542977,6.990076,2.671303,-2.711797e+06,-4.268995e+06,3.873143e+06,1589577040445,37.631667,-122.424972
90150,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,-0.000428,-2348.542977,16.376517,8.686505,-2.711797e+06,-4.268995e+06,3.873143e+06,1589577040445,37.631667,-122.424972
90151,Raw,1589577040445,13186908000000,NaN,-1273599071537517829,0.863317,18.450928,6.537852,9.715101,7,...,-0.009129,-2348.542977,7.696385,3.070332,-2.711797e+06,-4.268995e+06,3.873143e+06,1589577040445,37.631667,-122.424972
